# Pinecone

>[Pinecone](https://docs.pinecone.io/docs/overview) is a vector database with broad functionality.
```markdown
>[Pinecone](https://docs.pinecone.io/docs/overview) - это векторная база данных с широкими возможностями. В этом ноутбуке показано, как использовать функциональность, связанную с векторной базой данных `Pinecone`.

## Настройка

Для использования `PineconeVectorStore` вам сначала необходимо установить партнерский пакет, а также другие пакеты, используемые в этом ноутбуке.

In [1]:
%pip install -qU gigachain-pinecone pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


Примечание по миграции: если вы переходите с реализации Pinecone из `langchain_community.vectorstores`, вам, возможно, потребуется удалить зависимость `pinecone-client` версии 2 перед установкой `langchain-pinecone`, который зависит от `pinecone-client` версии 3.

### Авторизация

Создайте новый аккаунт в Pinecone или войдите в уже существующий, и создайте API-ключ для использования в этом ноутбуке.

In [32]:
import getpass
import os

from pinecone import Pinecone, ServerlessSpec

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

Если вы хотите автоматически отслеживать вызовы вашей модели, вы также можете установить API-ключ [LangSmith](https://docs.smith.langchain.com/), раскомментировав код ниже:

In [5]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

## Инициализация

Перед инициализацией нашего векторного хранилища давайте подключимся к индексу `Pinecone`. Если индекс с именем index_name не существует, он будет создан.

In [39]:
import time

index_name = "gigachain-test-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

Теперь можно иницилазировать векторное хранилище

```{=mdx}
import EmbeddingTabs from "@theme/EmbeddingTabs";

<EmbeddingTabs/>
```

In [40]:
# | output: false
# | echo: false
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

embeddings = GigaChatEmbeddings()

In [41]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

## Управление векторным хранилищем

После того как вы создали своё векторное хранилище, мы можем взаимодействовать с ним, добавляя и удаляя различные элементы.

### Добавляем элементы в векторное хранилище

Мы можем добавить документы с помощью функции `add_documents`.

In [42]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="Сегодня утром я ел шоколадные панкейки и омлет на завтрак.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="Прогноз погоды на завтра: облачно и пасмурно, с максимальной температурой 32 градуса.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Работаю над захватывающим новым проектом с GigaChain — приходите посмотреть!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Грабители ворвались в городской банк и украли 1 миллион долларов наличными.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Ух ты! Это был потрясающий фильм. Не могу дождаться, чтобы посмотреть его снова.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Стоит ли новая модель iPhone своей цены? Прочитайте этот обзор, чтобы узнать.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="Топ-10 лучших футболистов мира на данный момент.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="GigaGraph — лучшая платформа для создания приложений с состояниями и агентами!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="Рынок акций сегодня упал на 500 пунктов из-за опасений по поводу рецессии.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="У меня плохое предчувствие, что меня скоро удалят :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 28 Aug 2024 12:36:13 GMT', 'Content-Type': 'application/json', 'Content-Length': '103', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '483', 'x-pinecone-request-id': '4884807329039489018', 'x-envoy-upstream-service-time': '2', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 1024 does not match the dimension of the index 768","details":[]}


### Удаление элементов

In [25]:
vector_store.delete(ids=[uuids[-1]])

## Запросы

После того как ваше векторное хранилище создано и соответствующие документы добавлены, вам, скорее всего, захочется выполнять запросы к нему во время работы вашей цепочки или агента.

### Прямой запрос

Выполнить простой поиск по схожести можно следующим образом:

In [27]:
results = vector_store.similarity_search(
    "Работаю над захватывающим новым проектом с GigaChain — приходите посмотреть!",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Работаю над захватывающим новым проектом с GigaChain — приходите посмотреть! [{'source': 'tweet'}]
* GigaGraph — лучшая платформа для создания приложений с состояниями и агентами! [{'source': 'tweet'}]


#### Поиск похожих

Можно также выполнить поиск со скорингом:

In [30]:
results = vector_store.similarity_search_with_score(
    "Будет ли завтра жарко?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.615861] Прогноз погоды на завтра: облачно и пасмурно, с максимальной температурой 32 градуса. [{'source': 'news'}]


#### Другие методы поиска

Существуют и другие методы поиска (например, MMR), которые не перечислены в этом ноутбуке. Чтобы узнать о них всех, обязательно прочитайте [справочник API](https://python.langchain.com/v0.2/api_reference/pinecone/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html).


### Работа в режиме ретривера

Вы также можете превратить векторное хранилище в ретривер (retriever) для использования в цепочках

In [31]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)
retriever.invoke("Кража в банке", filter={"source": "news"})

[Document(metadata={'source': 'news'}, page_content='Грабители ворвались в городской банк и украли 1 миллион долларов наличными.')]

## Использование для генерации с дополнением извлечением данных

Для руководств по использованию этого векторного хранилища для генерации с дополнением извлечением данных (RAG) см. следующие разделы:

- [Учебные пособия: работа с внешними знаниями](https://python.langchain.com/v0.2/docs/tutorials/#working-with-external-knowledge)
- [Как это сделать: Вопросы и ответы с использованием RAG](https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag)
- [Концептуальные документы по извлечению](https://python.langchain.com/v0.2/docs/concepts/#retrieval)

## Ссылка на API

Для получения подробной документации по всем функциям и настройкам __ModuleName__VectorStore перейдите к справочнику API: https://python.langchain.com/v0.2/api_reference/pinecone/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html